# PCA and Friends Session 2
This is the second session, plannend for September 24, 2021. We're going to create our perform Principal Components for real genomic data and get our feed wet with some of the standard tools.

## Prequisites
In order to follow this session, we recommend that you first install miniconda (https://docs.conda.io/en/latest/miniconda.html). Once installed, you can use our environment file to easily install all the tools you need for this session.

First, clone the github repository if you haven't done so already:

```{bash}
git clone https://github.com/stschiff/exp_dat_reading_group_2021.git
cd exp_dat_reading_group_2021
```

If you have already cloned the repository you can make sure you have the latest update by running `git pull` inside of it.

Once miniconda has been installed, you can install the environment via:

```{bash}
conda env create -f session_2/environment.yml
conda activate PCA_session_2
```
That's it. Now you should have all the tools ready.

## Getting genotype data
We are going to use Poseidon (https://poseidon-framework.github.io/#/) to easily retrieve genotype data together with some useful annotation. The tool for accessing the Poseidon package repository is named `trident`, and if you followed the recommendation for installing the conda environment above, you should have it installed already. To make sure, check with `which trident` and `trident --version`.

`trident` is a command line tool to manage Poseidon packages. Here we'll use it to automatically download packages that we need for this session. You can list all available packages like so:

In [2]:
!trident list --remote --packages

Preparing output table
found 146 packages
.-----------------------------------------.----------------.
|                  Title                  | Nr Individuals |
:=========================================:================:
| 2010_RasmussenNature                    | 1              |
| 2012_KellerNatureCommunications         | 1              |
| 2012_MeyerScience                       | 6              |
| 2012_PattersonGenetics                  | 1036           |
| 2012_PickrellNatureCommunications       | 9              |
| 2014_FuNature                           | 1              |
| 2014_GambaNatureCommunications          | 13             |
| 2014_LazaridisNature                    | 1222           |
| 2014_MalaspinasCurrentBiology           | 2              |
| 2014_OlaldeNature                       | 1              |
| 2014_RaghavanNature                     | 6              |
| 2014_RaghavanScience                    | 4              |
| 2014_RasmussenNature                    |

Here we specifically need packages `2014_LazaridisNature`, which contains a lot of present-day individuals from around the world, and `2014_RaghavanScience`, which contains a famous 22,000 year old individual from Siberia. Let's fetch those packages and copy them into a local folder called `session_2/poseidon-repository` within this repository:

In [1]:
!mkdir poseidon-repository
!trident fetch -d poseidon-repository -f "*2014_LazaridisNature*,*2014_RaghavanScience*"

mkdir: poseidon-repository: File exists
Searching POSEIDON.yml files... 1 found
Initializing packages... 
> 1 
Packages loaded: 1
Determine requested packages... 2 requested and available
Comparing local and remote package state
Handling packages
2014_LazaridisNature                    local 1.1.2 = remote 1.1.2
2014_RaghavanScience                    > 14.3MB to download
> 5.6% > 11.2% > 16.8% > 22.4% > 28.0% > 33.6% > 39.2% > 44.8% > 50.3% > 55.9% > 61.5% > 67.1% > 72.7% > 78.3% > 83.9% > 89.5% > 95.1% > 100.0% 


Great, now we have those packages. You can checkout the files, e.g.:

In [9]:
!ls poseidon-repository/2014_LazaridisNature

2014_LazaridisNature.bed   2014_LazaridisNature.fam
2014_LazaridisNature.bib   2014_LazaridisNature.janno
2014_LazaridisNature.bim   POSEIDON.yml


And you can see three genotype files (`.bed`, `.bim` and `.fam`) and an annotation file ending with `.janno`.

You can also view lots of things about those packages using `trident`. For example:

In [2]:
!trident list --groups -d poseidon-repository/

Searching POSEIDON.yml files... 2 found
Initializing packages... 
> 1 > 2 
Packages loaded: 2
Preparing output table
found 177 groups/populations
.-----------------------------------------------.----------------------.----------------.
|                     Group                     |       Packages       | Nr Individuals |
:===============================================:======================:================:
| AA                                            | 2014_LazaridisNature | 12             |
| Abkhasian                                     | 2014_LazaridisNature | 9              |
| Afar.WGA                                      | 2014_LazaridisNature | 5              |
| Ain_Touta.WGA                                 | 2014_LazaridisNature | 3              |
| Albanian                                      | 2014_LazaridisNature | 6              |
| Aleut                                         | 2014_LazaridisNature | 7              |
| Algerian                                  

or:

In [4]:
!trident summarise -d poseidon-repository

Searching POSEIDON.yml files... 2 found
Initializing packages... 
> 1 > 2 
Packages loaded: 2
.------------------------.--------------------------------------------------------------.
|        Summary         |                            Value                             |
:========================:==============================================================:
| Nr Individuals         | 1226                                                         |
| Individuals            | ADR00057, ADR00059, ADR00060, ADR00061, ADR00064, ADR00065,… |
| Nr Groups              | 177                                                          |
| Groups                 | Spanish: 53, Turkish: 50, Ulchi: 25, Tubalar: 22, Chukchi: … |
| Nr Publications        | 2                                                            |
| Publications           | LazaridisNature2014, RaghavanScience2014                     |
| Nr Countries           | 66                                                           |
| Coun

OK, for further analysis we want to merge these two packages. In `trident` we can use the `forge` command for that. But we first need a population list to know what we like to extract and merge.